# Autofocus Checker
Built to evaluate autofocusing algorithms from Autofocusing Algorithm Selection in Computer Microscopy Yu Sun et al. 2005

### Imports

In [ ]:
from pycromanager import Core, Studio
import matplotlib.pyplot as plt
import numpy as np
import autofocus_tools_loci.autofocus_tools as af

### Init utility functions for microscope control

1. init_pycromanager() instantiates objects to connect to a microscope allowing for control

2. snap_image() takes a picture from the microscope and shapes it into a 2D or 3D ndarray based on if it's an RGB image or not

In [ ]:
def init_pycromanager():
    core = Core()
    studio = Studio()
    core.set_timeout_ms(20000)
    return core, studio
core,studio = init_pycromanager()

In [ ]:
def snap_image(core, rgb=True, flip_channel=True,brightfield_processing_function=None, show_image= False):
    
    core.snap_image()
    tagged_image = core.get_tagged_image()
    if rgb == True:
        pixels = np.reshape(
            tagged_image.pix,
            newshape=[tagged_image.tags["Height"], tagged_image.tags["Width"], 4],
            )
        pixels = pixels[:, :, 0:3]
        if flip_channel:
            pixels = np.flip(pixels, 2)
        if brightfield_processing_function:
            pixels = brightfield_processing_function(pixels)
    else:
        pixels = np.reshape(
            tagged_image.pix,
            newshape=[tagged_image.tags["Height"], tagged_image.tags["Width"]],
            )
    
    if show_image:
        plt.imshow(pixels)
    
    return pixels

### Set up variables and checks

In [ ]:
IS_RGB = True
if core.get_property('Core', 'Focus') != 'ZStage:Z:32':
    print(f"WARNING: FOCUS DEVICE SET TO: {core.get_property('Core', 'Focus')}")
z_start = core.get_position()
z_range = 25

In [ ]:
autofocus_methods = [
    af.threshold_absolute_gradient,
    af.squared_gradient,
    af.brenner_gradient,
    af.tenenbaum_gradient,
    af.sum_of_modified_laplace,
    af.energy_laplace,
    af.defocused_variance,
    af.normalized_variance,
    af.autocorrelation,
    af.standard_deviation_based_correlation,
    af.range_algorithm,
    af.thresholded_content,
    af.thresholded_pixel_count,
    af.image_power
]

### Shift z-stage and test all autofocus methods

In [ ]:
img_arr = []
for k in range(-25,+25,3):
    core.set_position(z_start+k)
    core.wait_for_device(core.get_focus_device())
    print(k,core.get_position())
    img_arr.append(snap_image(core,show_image=False))
    plt.plot(k, af.threshold_absolute_gradient(img_arr[-1].mean(2)),'or')